In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from backend.extractor.utils import parse_html, save_html
from backend.fetcher.fetcher import fetch_html, fetch_html_request
from backend.extractor.utils import clean_html, prepare_html, traverse_and_modify
from backend.extractor.task.container_extractor import ContainerExtractor
from backend.fetcher.fetcher import fetch_html, fetch_html_request
import asyncio

In [2]:
from backend.extractor.db import init_db

db = init_db()

In [9]:
url = "https://www.amazon.co.uk/b/?node=3028615031&ref_=Oct_d_odnav_d_376320011_2&pd_rd_w=39NlY&content-id=amzn1.sym.7414f21e-2c95-4394-9a75-8c1b3641bcea&pf_rd_p=7414f21e-2c95-4394-9a75-8c1b3641bcea&pf_rd_r=1J1AKE8E9C986540BGC7&pd_rd_wg=EiBjr&pd_rd_r=85013f62-4a9a-4353-9cd5-0ade0d82d28e"
html = await fetch_html(url)

/Users/minhphan/anaconda3/envs/thesis/lib/python3.12/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
2024-05-30 20:10:46,942 - fetcher.py: load_html_from_db(): 63:	Getting HTML from firebase
--------


In [42]:
html

<body class="a-aui_72554-c a-aui_a11y_2_750578-c a-aui_a11y_6_837773-c a-aui_a11y_sr_678508-t1 a-aui_amzn_img_959719-c a-aui_amzn_img_gate_959718-c a-aui_killswitch_csa_logger_372963-c a-aui_pci_risk_banner_210084-c a-aui_rel_noreferrer_noopener_309527-c a-aui_template_weblab_cache_333406-c a-aui_tnr_v2_180836-c a-meter-animate" style="padding-bottom: 405px;"><div id="a-page"><!-- sp:end-feature:start-body -->
<!-- sp:feature:csm:body-open -->



<!-- sp:end-feature:csm:body-open -->
<!-- sp:feature:cookie-consent-banner -->
<span class="a-declarative" data-action="sp-cc" data-csa-c-func-deps="aui-da-sp-cc" data-csa-c-id="yynuhl-vmj806-uw7osg-guk66d" data-csa-c-type="widget" data-sp-cc='{"customerId":"null","rejectAllAction":"/privacyprefs/sp/consent/v2/rejectall","sessionId":"259-4080907-8365646","userId":"259-4080907-8365646"}'><form action="/privacyprefs/sp/consent/v2/acceptall" id="sp-cc" method="post"><div class="sp-cc-text"><h4 class="a-spacing-small">Select your cookie preferenc

In [10]:
html = prepare_html(html)

2024-05-30 20:10:47,639 - utils.py: parse_html(): 76:	Parsing HTML string to BeautifulSoup object
--------
2024-05-30 20:10:47,852 - utils.py: clean_html(): 14:	Cleaning HTML, removing all path that does not contains navigable string
--------


body -> div -> a -> Skip to main content
body -> div -> span -> form -> div -> h4 -> Select your cookie preferences
body -> div -> span -> form -> div -> p -> a -> Cookie notice
body -> div -> span -> form -> div -> p -> a -> Cookie notice
body -> div -> span -> form -> div -> p -> a -> 103 third parties
body -> div -> span -> form -> div -> p -> a -> Cookie preferences
body -> div -> span -> form -> div -> p -> a -> Privacy notice
body -> div -> span -> form -> div -> div -> div -> div -> Sorry, there was a problem saving your preferences. Try again.
body -> div -> span -> form -> div -> span -> span -> span -> Accept
body -> div -> span -> form -> div -> span -> span -> button -> Decline
body -> div -> span -> form -> div -> div -> a -> Customise
body -> div -> header -> div -> div -> div -> div -> a -> span -> .co.uk
body -> div -> header -> div -> div -> div -> div -> span -> a -> div -> span -> Deliver to
body -> div -> header -> div -> div -> div -> div -> span -> a -> div -> spa

In [27]:
example_container = {
    "product": "Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder",
    "price": "£24.99",
    "rating": "4.6 out of 5",
}

In [62]:
extractor = ContainerExtractor(str(html), example_container)

2024-05-31 01:10:23,479 - utils.py: parse_html(): 76:	Parsing HTML string to BeautifulSoup object
--------


In [45]:
print(extractor.lca.prettify())

<div class="a-section a-spacing-small puis-padding-left-small puis-padding-right-small">
 <div class="a-section a-spacing-none a-spacing-top-small s-title-instructions-style" data-cy="title-recipe">
  <h2 class="a-size-mini a-spacing-none a-color-base s-line-clamp-4">
   <a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/Sculpture-Decorative-Figurine-Entrance-Decoration/dp/B0922CCRCN/ref=sr_1_1?c=ts&amp;dib=eyJ2IjoiMSJ9.6UJPju-fnbhlzsxcJI7h6ccVXSCninW7VtZi4Ht9yxEug-MLfqbhTg4cilFuaCcS4hsvM65Z7X7FQZON2z5a-43OrPqB3jEqs6IoFe4BXUl-AVk3h3Y8tAn2eSLe7oMFWQH7KC5rGKTvN5xIU9Cw78fbYiwAoCN3nyxAbvrMt2uryvDVFRpnjWttDJceGvFJI6xM1vxQq4tn6FiT3lTULT9yh3xqdOYydoLyOoBrglU7mnMJagF5qmhH_01JU8VPJOmepKbVFYWbICnNlkdi0ssVktsYrydQYdBOJ5VBjiM.1PFRfXlDZ_V0uhPX-SJ3W8ZDiwXIDvp5hmw9CEVEWQk&amp;dib_tag=se&amp;keywords=Decorative+Home+Accessories&amp;qid=1717073806&amp;s=kitchen&amp;sr=1-1&amp;ts_id=3028615031">
    <span class="a-size-base-plus a-color-base a-text-normal">

In [51]:
async def find_similar_containers(extractor):
    "Uses the path of an example container, to find other similar containers"
    selector = extractor.path_to_lca.replace(" ", " > ")

    return extractor.soup.select(selector)

similar_containers = await find_similar_containers(extractor)
similar_containers

50

In [61]:
extractor.template_structure

{'product': 'div h2 a span',
 'price': 'div div span',
 'rating': 'div div span span a i span'}

In [60]:
from backend.extractor.task.container_extractor import filter_non_empty_dicts


async def extract_items_in_container(container, template_structure) -> dict:
    """Receives a container and a template structure, returns a dictionary populated with content in the corresponding keys specified by the template structure
    Example structure:
    extracted_item = {"title": "ABC", "description": "xyz"}
    """
    res_dict = {}
    for key, value in template_structure.items():
        string_values = []
        selector = value.replace(" ", " > ")
        content_tags = container.select(selector)
        string_values += (
            [content_tag.get_text().strip() for content_tag in content_tags]
            if content_tags is not None
            else ""
        )
        res_dict[key] = string_values
    return res_dict

contents = []
for container in similar_containers:
    content = await extract_items_in_container(container, extractor.template_structure)
    contents.append(content)

filtered = await filter_non_empty_dicts(contents)
filtered

[{'product': [],
  'price': ['Amazon\'s Choicefor "Decorative Home Accessories"'],
  'rating': []},
 {'product': ['Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder Modern Art Statue Home Desktop Ornament Decoration(Black)'],
  'price': ['4.6 out of 5 stars',
   '75',
   'No featured offers available',
   '£24.99',
   '(2 new offers)',
   ''],
  'rating': ['4.6 out of 5 stars']},
 {'product': ['Creative Scents Dublin Decorative Tray and Orbs Balls Set - Centerpiece Bowl with Balls for Dining Room Table, Rustic Decorated Spheres Kit for Living Room Coffee Table, Gift Boxed (Brushed Silver)'],
  'price': ['4.7 out of 5 stars',
   '2,448',
   'No featured offers available',
   '£48.70',
   '(2 new offers)',
   ''],
  'rating': ['4.7 out of 5 stars']},
 {'product': ['Coasters for Drinks, Wooden Heart Coasters Set of 6 Farmhouse Coasters with Holder Funny Coasters for Coffee Table Protection Housewarming Gifts for New Home Decoration, 4 In

In [63]:
found_contents, _ = await extractor.container_extract_run_task()

2024-05-31 01:12:07,245 - container_extractor.py: prepare_extract(): 41:	Preparing extract
--------
2024-05-31 01:12:07,248 - extractor.py: find_string_tag(): 14:	Found string in soup at: Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder Modern Art Statue Home Desktop Ornament Decoration(Black)
--------
2024-05-31 01:12:07,252 - extractor.py: find_string_tag(): 14:	Found string in soup at: Â£24.99
--------
2024-05-31 01:12:07,255 - extractor.py: find_string_tag(): 14:	Found string in soup at: 4.6 out of 5 stars
--------
2024-05-31 01:12:07,293 - container_extractor.py: find_container_from_examples(): 88:	Found container tag: div
                    Found path to container: body -> div -> div -> div -> div -> div -> span -> div -> div -> div -> div -> span -> div -> div -> div
--------
2024-05-31 01:12:07,294 - container_extractor.py: set_up_container_template_structure(): 104:	Setting up template structure for container extraction
---

TypeError: sequence item 4: expected str instance, NoneType found

In [65]:
from backend.extractor.extractor import find_string_tag


find_string_tag(extractor.soup, "£24.99")

2024-05-31 01:14:53,288 - extractor.py: find_string_tag(): 14:	Found string in soup at: None
--------


In [31]:
found_contents

[{'product': [],
  'price': ['Amazon\'s Choicefor "Decorative Home Accessories"',
   'Amazon\'s Choicefor "Decorative Home Accessories"',
   'Amazon\'s Choicefor "Decorative Home Accessories"',
   "Amazon's Choice",
   "Amazon's Choice",
   "Amazon's",
   'Choice',
   'for "Decorative Home Accessories"'],
  'rating': []},
 {'product': [],
  'price': ['Amazon\'s Choicefor "Decorative Home Accessories"',
   "Amazon's Choice",
   "Amazon's Choice",
   "Amazon's",
   'Choice',
   'for "Decorative Home Accessories"'],
  'rating': []},
 {'product': [], 'price': [''], 'rating': []},
 {'product': ['Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder Modern Art Statue Home Desktop Ornament Decoration(Black)'],
  'price': ['Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder Modern Art Statue Home Desktop Ornament Decoration(Black)',
   '4.6 out of 5 stars',
   '4.6 out of 5 stars',
   '4.6 out of

In [38]:
containers = await extractor.find_similar_containers()

2024-05-30 20:30:48,276 - container_extractor.py: find_similar_containers(): 110:	Getting list of similar containers
--------


In [41]:
containers[236]

<div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-spacing-small s-height-equalized puis puis-v2zbxj9wv0qyon2055dia086ji7" style="padding-top: 0px !important;"><span class="rush-component" data-component-type="s-product-image" data-render-id="r37q8vah3xfh3z2bk03c30kkw6d" data-version-id="v2zbxj9wv0qyon2055dia086ji7"><a class="a-link-normal s-no-outline" href="/Carousel-Home-Gifts-Stacking-Decoration/dp/B084YZ2SQ7/ref=sr_1_16?c=ts&amp;dib=eyJ2IjoiMSJ9.6UJPju-fnbhlzsxcJI7h6ccVXSCninW7VtZi4Ht9yxEug-MLfqbhTg4cilFuaCcS4hsvM65Z7X7FQZON2z5a-43OrPqB3jEqs6IoFe4BXUl-AVk3h3Y8tAn2eSLe7oMFWQH7KC5rGKTvN5xIU9Cw78fbYiwAoCN3nyxAbvrMt2uryvDVFRpnjWttDJceGvFJI6xM1vxQq4tn6FiT3lTULT9yh3xqdOYydoLyOoBrglU7mnMJagF5qmhH_01JU8VPJOmepKbVFYWbICnNlkdi0ssVktsYrydQYdBOJ5VBjiM.1PFRfXlDZ_V0uhPX-SJ3W8ZDiwXIDvp5hmw9CEVEWQk&amp;dib_tag=se&amp;keywords=Decorative+Home+Accessories&amp;qid=1717073806&amp;s=kitchen&amp;s

In [37]:
found_contents[234]

{'product': ['Colorful Water Beads Crystal Gel Beads Water Beads 50000 Gel Soil Water Floating Candles Making, Wedding Party Decoration Relief as Vase Filler Decor Decorative Home Accessories Plants and Craft'],
 'price': ['Colorful Water Beads Crystal Gel Beads Water Beads 50000 Gel Soil Water Floating Candles Making, Wedding Party Decoration Relief as Vase Filler Decor Decorative Home Accessories Plants and Craft',
  '3.3 out of 5 stars',
  '3.3 out of 5 stars',
  '3.3 out of 5 stars',
  '26',
  '26',
  '100+ bought in past month',
  'No featured offers available',
  '£5.89',
  '(1 new offer)',
  ''],
 'rating': ['3.3 out of 5 stars']}

In [64]:
extractor.lca
selector = extractor.path_to_lca
print(selector)

div div div div div div section div div div div div div div div div article div div div


In [58]:
selector.replace(" ", " > ")
found = extractor.soup.select(selector)

In [66]:
found

[<div class="jeg_post_category"><a class="category-activities-programs" href="https://iuoss.com/category/activities-programs/">Activities - Programs</a></div>,
 <div class="jeg_post_info">
 <h2 class="jeg_post_title">
 <a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/">Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024</a>
 </h2>
 <div class="jeg_post_meta"><div class="jeg_meta_date"><a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/"> 27/05/2024</a></div></div>
 </div>,
 <div class="jeg_post_meta"><div class="jeg_meta_date"><a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/"> 27/05/2024</a></div></div>,
 <div class="jeg_meta_date"><a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/"> 27/05/2024</a></div>,
 <div class="jeg_post_category"><a class="category-thongbao" href="https://iuoss.com/category/thongbao/">Announcements</a></div>,
 <div class="jeg_post_

In [84]:
async def find_similar_containers(extractor):
    selector = extractor.path_to_lca.replace(" ", " > ")
    print(selector)
    return extractor.soup.select(selector)

In [85]:
set_up_container_task = asyncio.create_task(extractor.set_up_container_template_structure())
get_similar_containers_task = asyncio.create_task(
    find_similar_containers(extractor)
)
template_structure, similar_containers = await asyncio.gather(
set_up_container_task, get_similar_containers_task
)

2024-05-29 12:14:40,845 - container_extractor.py: set_up_container_template_structure(): 97:	Setting up template structure for container extraction
--------


<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/">TB V/v nhắc nhở sinh viên về công tác phòng, chống tội phạm, vi phạm pháp luật, HIV/AIDS, ma túy, mại dâm trong ngành giáo dục năm 2024</a>
h3
<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/"> 15/03/2024</a>
div
div
div > div > div > div > div > div > section > div > div > div > div > div > div > div > div > article > div


In [86]:
found_contents = []
tasks = []
for container in similar_containers:
    tasks.append(
        asyncio.create_task(
            extractor.extract_items_in_containers(
                container=container, template_structure=template_structure
            )
        )
    )
found_contents = await asyncio.gather(*tasks)

In [132]:
await extractor.prepare_extract()
unstructured_content = await extractor.extract_unstructured_text_from_containers()
unstructured_content

2024-05-29 15:07:25,804 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a data-itm-added="1" data-itm-source="#vn_source=Folder-TheGioi_QuanSu&amp;vn_campaign=Stream&amp;vn_medium=Item-2&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-2" data-thumb="1" href="https://vnexpress.net/cho-robot-trung-quoc-bat-nhay-ban-sung-khi-dien-tap-o-campuchia-4751792.html" title="ChÃ³ robot Trung Quá»c báº­t nháº£y, báº¯n sÃºng khi diá»n táº­p á» Campuchia">
    ChÃ³ robot Trung Quá»c báº­t nháº£y, báº¯n sÃºng khi diá»n táº­p á» Campuchia </a>
--------
2024-05-29 15:07:25,815 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a data-itm-added="1" data-itm-source="#vn_source=Folder-TheGioi_QuanSu&amp;vn_campaign=Stream&amp;vn_medium=Item-2&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-2" data-thumb="1" href="https://vnexpress.net/cho-robot-trung-quoc-bat-nhay-ban-sung-khi-dien-tap-o-campuchia-4751792.html" title="ChÃ³ robot Trung Quá»c báº­t nháº£y, bá

<article class="item-news thumb-left item-news-common">
<div class="thumb-art">
<a class="thumb thumb-5x3" data-itm-added="1" data-itm-source="#vn_source=Folder-TheGioi_QuanSu&amp;vn_campaign=Stream&amp;vn_medium=Item-2&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-2" data-thumb="1" href="https://vnexpress.net/cho-robot-trung-quoc-bat-nhay-ban-sung-khi-dien-tap-o-campuchia-4751792.html" title="Chó robot Trung Quốc bật nhảy, bắn súng khi diễn tập ở Campuchia">
<video autoplay="" class="lazy thumb-gif error" data-ll-status="error" itemprop="contentUrl" loop="" muted="" playsinline="" poster="https://i1-vnexpress.vnecdn.net/2024/05/29/1234-1716954081-6963-1716954434.gif?w=300&amp;h=180&amp;q=100&amp;dpr=1&amp;fit=crop&amp;s=4kpuoQbTk1l9HGgfneJsGQ&amp;t=image" preload="auto" src="https://i1-vnexpress.vnecdn.net/2024/05/29/1234-1716954081-6963-1716954434.gif?w=300&amp;h=180&amp;q=100&amp;dpr=1&amp;fit=crop&amp;s=ZnNwYk5HxlLhr20XrhfXdQ&amp;t=video" style="transform: translateX(-50%);

['[if IE 9]><video style="display: none;"><![endif][if IE 9]></video><![endif]\nUkraine phóng \'tên lửa ATACMS\' tập kích sân bay Nga ở Lugansk Ukraine phóng tên lửa được cho là ATACMS vào sân bay Nga ở Lugansk, gây ra nhiều vụ nổ, có thể đã gây thiệt hại cho một radar tầm xa. ',
 '\nChó robot Trung Quốc bật nhảy, bắn súng khi diễn tập ở Campuchia Mẫu chó robot mới của Trung Quốc có thể cử động linh hoạt và được gắn súng trường để tấn công khi tham gia diễn tập chung với Campuchia.28',
 '[if IE 9]><video style="display: none;"><![endif][if IE 9]></video><![endif]\nUkraine lần đầu công bố ảnh tên lửa \'quái vật Frankenstein\' Quân đội Ukraine lần đầu công bố hình ảnh bệ phóng thời Liên Xô lắp ống phóng đạn tên lửa Mỹ để tạo thành tổ hợp phòng không kiểu "quái vật Frankenstein".10',
 '[if IE 9]><video style="display: none;"><![endif][if IE 9]></video><![endif]\nUkraine nói Nga sắp điều thêm hàng trăm nghìn quân đến biên giới Bộ trưởng Quốc phòng Ukraine cho biết nửa triệu quân Nga đang t

In [108]:
unextracted_contents = []
for container in similar_containers:
    contents = container.find_all(string=True, recursive=True)
    filtered = [string for string in contents if string.strip()]
    unextracted_contents.append(''.join(filtered))

In [122]:
structured, unstructured = await extractor.container_extract_run_task()

2024-05-29 14:53:20,509 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/">TB V/v nháº¯c nhá» sinh viÃªn vá» cÃ´ng tÃ¡c phÃ²ng, chá»ng tá»i pháº¡m, vi pháº¡m phÃ¡p luáº­t, HIV/AIDS, ma tÃºy, máº¡i dÃ¢m trong ngÃ nh giÃ¡o dá»¥c nÄm 2024</a>
--------
2024-05-29 14:53:20,512 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/"> 15/03/2024</a>
--------
2024-05-29 14:53:20,514 - container_extractor.py: find_container_from_examples(): 87:	Found container tag: div
                    Found path to container: div -> div -> div -> div -> div -> div -> section -> div -> div -> div -> div -> div -> div -> div -> div -> article -> div
--------
2024-05-29 

<div class="jeg_postblock_content">
<h3 class="jeg_post_title">
<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/">TB V/v nhắc nhở sinh viên về công tác phòng, chống tội phạm, vi phạm pháp luật, HIV/AIDS, ma túy, mại dâm trong ngành giáo dục năm 2024</a>
</h3>
<div class="jeg_post_meta"><div class="jeg_meta_author"><span class="by">by</span> <a href="https://iuoss.com/author/lttmai/">lttmai</a></div><div class="jeg_meta_date"><a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/"> 15/03/2024</a></div></div>
<div class="jeg_post_excerpt">
<p>Hiện nay, tội phạm, vi phạm pháp luật về ma túy, mại dâm đang là vấn đề đáng lo ngại...</p>
</div>
</div>
article
div
div
div
div
div
div
div
div
section
div
div
div
div
div
div
<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-

In [123]:
structured

[{'article': [],
  'date': ['',
   'Announcements',
   'Chương trình học bổng Vừ A Dính năm học 2023-2024',
   'vnthieu',
   '28/05/2024']},
 {'article': [], 'date': ['']},
 {'article': [],
  'date': ['Announcements',
   'Chương trình học bổng Vừ A Dính năm học 2023-2024',
   'vnthieu',
   '28/05/2024']},
 {'article': [], 'date': ['Announcements']},
 {'article': [],
  'date': ['Chương trình học bổng Vừ A Dính năm học 2023-2024',
   'vnthieu',
   '28/05/2024']},
 {'article': [], 'date': ['vnthieu', '28/05/2024']},
 {'article': [], 'date': ['vnthieu']},
 {'article': [], 'date': ['28/05/2024']},
 {'article': [],
  'date': ['',
   'Activities - Programs',
   'Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024',
   '27/05/2024']},
 {'article': [], 'date': ['']},
 {'article': [],
  'date': ['Activities - Programs',
   'Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024',
   '27/05/2024']},
 {'article': [], 'date': ['Activities - Programs']},
 {'article': [],
  'date

In [124]:
unstructured

['AnnouncementsChương trình học bổng Vừ A Dính năm học 2023-2024byvnthieu 28/05/2024',
 '',
 'AnnouncementsChương trình học bổng Vừ A Dính năm học 2023-2024byvnthieu 28/05/2024',
 'Announcements',
 'Chương trình học bổng Vừ A Dính năm học 2023-2024byvnthieu 28/05/2024',
 'byvnthieu 28/05/2024',
 'byvnthieu',
 ' 28/05/2024',
 'Activities - ProgramsTriển lãm thuộc Diễn đàn Công nghệ La French Tech 2024 27/05/2024',
 '',
 'Activities - ProgramsTriển lãm thuộc Diễn đàn Công nghệ La French Tech 2024 27/05/2024',
 'Activities - Programs',
 'Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024 27/05/2024',
 ' 27/05/2024',
 ' 27/05/2024',
 'AnnouncementsTHÔNG BÁO v/v hỗ trợ trực tuyến cho sinh viên tự chấm điểm rèn luyện 23/05/2024',
 '',
 'AnnouncementsTHÔNG BÁO v/v hỗ trợ trực tuyến cho sinh viên tự chấm điểm rèn luyện 23/05/2024',
 'Announcements',
 'THÔNG BÁO v/v hỗ trợ trực tuyến cho sinh viên tự chấm điểm rèn luyện 23/05/2024',
 ' 23/05/2024',
 ' 23/05/2024',
 'Activi

In [162]:



clean_found_contents = await filter_non_empty_dicts(found_contents)

In [163]:
clean_found_contents

[{'title': ['Hà Lan muốn cung cấp linh kiện để lắp Patriot cho Ukraine'],
  'details': ['Bộ trưởng Ollongren cho biết Hà Lan không muốn gửi một tổ hợp Patriot hoàn chỉnh cho Ukraine, thay vào đó cùng đối tác góp linh kiện lắp ráp.',
   '2']},
 {'title': ['Khoảnh khắc tên lửa đẩy Triều Tiên nổ tung trên không'],
  'details': ['Nhật, Hàn công bố cảnh quay từ mặt đất cho thấy tên lửa đẩy mang vệ tinh trinh sát quân sự Malligyong-1 của Triều Tiên phát nổ giữa không trung.',
   '']},
 {'title': ['Châu Âu thiếu thuốc nổ chế đạn pháo, tên lửa'],
  'details': ['Các nhà máy vũ khí châu Âu không đủ thuốc nổ để chế tạo đạn pháo và tên lửa, trong khi nỗ lực mở rộng nguồn cung vấp phải nhiều trở ngại.',
   '31']},
 {'title': ['Loạt vũ khí dẫn đường Mỹ bị Nga vô hiệu hóa ở Ukraine'],
  'details': ['Nhiều loại vũ khí chính xác cao Mỹ viện trợ cho Ukraine bị vô hiệu hóa, không thể đánh trúng mục tiêu do tác chiến điện tử Nga.',
   '46']},
 {'title': ['Thiết giáp Bradley cứu lính Ukraine giữa làn đạn']